In [ ]:
from calitp.storage import get_fs
fs = get_fs()

In [ ]:
from tqdm.notebook import tqdm
files = fs.expand_path('gs://calitp-gtfs-schedule-raw/schedule/', recursive=True)
files = [file for file in files if "ts" in file and fs.stat(file)["type"] != "directory" and "2022-07-15" not in file]
len(files)

In [ ]:
paths = [(path, *path.split("/")) for path in files]
paths[:5]

In [ ]:
%pip install pendulum

In [ ]:
import pendulum
from datetime import datetime
moves = []
for og_path, bucket, table, dt, base64url, time, filename in paths:
    pdt = pendulum.parse(dt.replace("dt=", ""), exact=True)
    assert isinstance(pdt, pendulum.Date)
    ptime = pendulum.parse(time.replace("time=", ""), exact=True)
    assert isinstance(ptime, pendulum.Time)
    ts = pendulum.instance(datetime.combine(pdt, ptime))
    assert isinstance(ts, pendulum.DateTime)
    new_path = "/".join([bucket, table, dt, base64url, f"ts={ts.to_iso8601_string()}", filename])
    moves.append((og_path, new_path))
moves[:5]

In [ ]:
for og_path, new_path in tqdm(moves):
    #print(og_path, new_path)
    #break
    fs.mv(og_path, new_path)

In [ ]:
import json
notices = {
    filename: json.loads(fs.cat(f"gs://{filename}").decode())
    for filename in tqdm(files)
}
notices["gtfs-data/schedule/processed/2022-06-27_101_0/validation_report.json"].keys()

In [ ]:
d = notices["gtfs-data/schedule/processed/2022-06-27_101_0/validation_report.json"]
d["notices"][0]

In [ ]:
all_keys = []

for name, file in notices.items():
    for notice in file["notices"]:
        for subnotice in notice["notices"]:
            all_keys.append((name, subnotice.keys()))
all_keys[0]

In [ ]:
[stuff for stuff in all_keys if 'columnName' in stuff[1]]

In [ ]:
rts = fs.glob('gs://gtfs-data/rt/2022-06-28T14:00:39/**')
rts = [fs.info(file) for file in rts if file.endswith('url')]
sorted(rts, key=lambda f: f['size'], reverse=True)[:5]